# NLP

# 2. Трансформеры: база

<img src="attachements/second/attn.png" alt="Attention is all you need" width="500"/>

## План занятий

### 1. ЧБДТ: Что Было До Трансформеров
   1. При чём тут нейро-лингвистическое программирование?
   2. Почему вектора?
   3. Счётные методы: CountVectorizer, TF-IDF, добавляем контекст 
   4. Дистрибутивная семантика: Word2vec, FastText
   5. Рекуррентные нейронные сети: LSTM, GRU
### 2. Трансформеры: база
   1. Мотивация
   2. Attention из all you need?: виды attention, интуиция, реализация
   3. Архитектура Transformer: эмбеддинги, энкодер, декодер
### 3. Трансформеры: на волне хайпа
   1. BERT (NLU)
   2. GPT (Language Modeling)
   3. T5 (Seq2seq)

## Мотивация

## Прошлые модели

### W2v and family

<img src="attachements/first/w2v_properties.jpg" width="800"/>

### RNN and family

<img src="attachements/first/rnn_basic.png" width="800"/>

### Проблемы

- Out of vocabulary
- Слабое понимание контекста (в RNN чуть получше)
- Забывание (даже в LSTM)
- Нет хороших претрейнутых моделей (как в CV; хотя w2v и ELMO поспорят)

<img src="attachements/second/rnn_vs_transformers.png" width="800"/>

## Attention

Attention (внимание) - на что нам нужно обратить внимание, чтобы понять, что происходит?

<img src="attachements/second/coreference.png" width="1000"/>

Аттеншн помогает решить проблему.

<img src="attachements/second/attn_vis.png" width="1000"/>

- Принимает на вход: две последовательности (возможно, одинаковые).
- Даёт на выход: вектор "внимания", значения которого показывают важность каждого слова.

### Lazy self-attention

Self-attention - принимает на вход два одинаковых предложения: делает attention сам на себя.

Посмотрим пример из жизни поисковой системы по музыке.

<img src="attachements/second/self_attn_1.png" width="600"/>

Умножаем вектор нужного слова на остальные вектора.

<img src="attachements/second/self_attn_2.png" width="600"/>

# Почему это ок? Про косинусное расстояние и иннер продакт

Берём софтмакс, чтобы получить веса слов.

<img src="attachements/second/self_attn_3.png" width="600"/>

Получаем итоговый вектор слова, взяв взвешенное среднее всех векторов с полученными коэффициентами.

<img src="attachements/second/self_attn_4.png" width="600"/>

<img src="attachements/second/self_attn_5.png" width="600"/>

In [1]:
from navec import Navec
import torch
import numpy as np

navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

In [2]:
text = "включи пожалуйста группу руки вверх"

embeddings = list(map(navec.get, text.split()))
embeddings = torch.from_numpy(np.stack(embeddings))
embeddings.shape

torch.Size([5, 300])

In [3]:
scores = embeddings @ embeddings.T
scores

tensor([[32.5143,  9.6992,  2.3823,  0.7420, -1.0193],
        [ 9.6992, 36.3740,  2.5938, 11.2488,  6.6931],
        [ 2.3823,  2.5938, 35.1974,  3.1319,  4.3334],
        [ 0.7420, 11.2488,  3.1319, 32.9065, 13.9943],
        [-1.0193,  6.6931,  4.3334, 13.9943, 35.9186]])

In [4]:
scores_normalized = scores.softmax(dim=1)
scores_normalized

tensor([[1.0000e+00, 1.2346e-10, 8.2006e-14, 1.5902e-14, 2.7324e-15],
        [2.6020e-12, 1.0000e+00, 2.1353e-15, 1.2254e-11, 1.2875e-13],
        [5.6052e-15, 6.9255e-15, 1.0000e+00, 1.1861e-14, 3.9441e-14],
        [1.0743e-14, 3.9280e-10, 1.1723e-13, 1.0000e+00, 6.1167e-09],
        [9.0800e-17, 2.0302e-13, 1.9175e-14, 3.0088e-10, 1.0000e+00]])

In [5]:
scores_normalized[:, 0].view(-1, 1)

tensor([[1.0000e+00],
        [2.6020e-12],
        [5.6052e-15],
        [1.0743e-14],
        [9.0800e-17]])

In [6]:
first_emb = (embeddings * scores_normalized[:, 0].view(-1, 1)).sum(dim=0)
first_emb

tensor([ 8.3382e-01, -2.1397e-01,  2.0033e-01, -2.3234e-02, -1.6352e-01,
        -3.1473e-01, -2.7189e-01,  1.7513e-01,  4.2780e-01,  3.4569e-01,
        -1.8912e-01, -2.5101e-02,  4.4553e-02, -6.3359e-01, -3.4898e-01,
         1.6900e-02, -2.2920e-02, -1.6965e-01,  1.5217e-01, -2.2922e-01,
        -8.7308e-01,  3.4396e-01, -6.2545e-03,  1.3233e-01, -4.0724e-01,
         3.1201e-01,  1.0364e-01, -2.7280e-01, -7.5732e-02,  2.6447e-01,
         1.0375e-01, -1.3977e-01, -5.4358e-01, -1.0175e-01,  2.3113e-02,
        -1.7922e-01, -1.4222e-01,  2.4688e-01,  7.0777e-01, -1.8071e-01,
        -9.9012e-02,  7.4856e-01,  3.4141e-01, -1.3784e-01,  2.0613e-02,
         2.7204e-01, -2.9992e-02,  2.3393e-01,  2.8204e-01,  1.0993e-01,
        -7.0768e-01, -2.0828e-03,  6.4351e-01, -8.2678e-02, -5.6086e-02,
        -4.7781e-03,  7.6802e-03,  2.0830e-01,  2.6347e-01,  4.4920e-02,
        -2.3996e-01, -2.0465e-01, -2.7938e-03,  1.9067e-01, -2.5622e-01,
        -8.0669e-02, -4.3525e-01, -3.1192e-01, -1.4

In [7]:
first_emb == embeddings[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [8]:
embeddings.shape

torch.Size([5, 300])

In [9]:
scores_normalized.shape

torch.Size([5, 5])

In [10]:
repeated_embeddings = embeddings.repeat(embeddings.shape[0], 1, 1)
repeated_embeddings.shape

torch.Size([5, 5, 300])

In [11]:
repeated_embeddings[:, 0, :]

tensor([[ 0.8338, -0.2140,  0.2003,  ...,  0.5876, -0.3866, -0.1288],
        [ 0.8338, -0.2140,  0.2003,  ...,  0.5876, -0.3866, -0.1288],
        [ 0.8338, -0.2140,  0.2003,  ...,  0.5876, -0.3866, -0.1288],
        [ 0.8338, -0.2140,  0.2003,  ...,  0.5876, -0.3866, -0.1288],
        [ 0.8338, -0.2140,  0.2003,  ...,  0.5876, -0.3866, -0.1288]])

In [12]:
permuted_embeddings = repeated_embeddings.permute(2, 0, 1)
permuted_embeddings.shape

torch.Size([300, 5, 5])

In [13]:
embeddings_attented = (permuted_embeddings * scores_normalized).sum(dim=2).permute(1, 0)
embeddings_attented.shape

torch.Size([5, 300])

In [14]:
embeddings_attented == embeddings

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

И тут пришло осознание...

In [44]:
(scores_normalized @ embeddings) == embeddings_attented

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [41]:
def lazy_self_attn(embeddings: torch.Tensor) -> torch.Tensor:
    """embeddings (seq_len, emb_dim)"""
    scores = embeddings @ embeddings.T  # (seq_len, seq_len)
    scores_normalized = scores.softmax(dim=1)  # (seq_len, seq_len)
    
    # repeated_embeddings = embeddings.repeat(embeddings.shape[0], 1, 1)  # (seq_len, seq_len, emb_dim)
    # permuted_embeddings = repeated_embeddings.permute(2, 0, 1)  # (emb_dim, seq_len, seq_len)

    # embeddings_attented = (permuted_embeddings * scores_normalized).sum(dim=2).permute(1, 0)  # (seq_len, emb_dim)

    embeddings_attented = scores_normalized @ embeddings

    return embeddings_attented

In [42]:
lazy_self_attn(embeddings).shape

torch.Size([5, 300])

<img src="attachements/second/self_attn_full.png" width="500"/>

**Весёлое упражнение:** батчизовать! Принимать на вход и возвращать `(B, seq_len, emb_dim)`. Без циклов, конечно.

Проблема: сильно слабый, не годится для хорошего понимания контекста.

Для чего он может быть хорош?

### Scaled dot-product Attention

- Query - что хотим найти? (токен, для которого хотим посчитать аттеншн)
- Key - что во мне содержится? (токены, на которые будем смотреть)
- Value - какая информация тебе нужна? (значения, которые будем получать)

<img src="attachements/second/attn_intuition.png" width="1400"/>

Query, Key, Value - обучаемые матрицы (по сути, обычные линейные слои).

<img src="attachements/second/attn_matrix_base.png" width="600"/>

Итоговая формула:

<img src="attachements/second/attention_matrices.webp" width="800"/>

<img src="attachements/second/attn_formula.png" width="800"/>

## Explain scaling!!!

In [55]:
torch.tensor([1.,2.]).softmax(dim=0)

tensor([0.2689, 0.7311])

In [17]:
text2 = "руки вверх восемнадцать мне уже"

embeddings2 = list(map(navec.get, text2.split()))
embeddings2 = torch.from_numpy(np.stack(embeddings2))
embeddings2.shape

torch.Size([5, 300])

In [18]:
embeddings @ embeddings2.T

tensor([[ 0.7420, -1.0193,  0.5002,  1.5011, -0.6839],
        [11.2488,  6.6931,  5.1217, 15.4657,  6.2644],
        [ 3.1319,  4.3334,  4.2619,  3.9792,  4.6229],
        [32.9065, 13.9943,  1.9484, 10.9348,  8.9146],
        [13.9943, 35.9186,  3.1689,  4.2400,  5.2509]])

In [19]:
from torch import nn

In [22]:
torch.sqrt(torch.tensor(12))

tensor(3.4641)

In [45]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, emb_size):
        super().__init__()

        self.emb_size = emb_size
        self.scale = torch.sqrt(torch.tensor(emb_size))

        self.q_w = nn.Linear(emb_size, emb_size)
        self.k_w = nn.Linear(emb_size, emb_size)
        self.v_w = nn.Linear(emb_size, emb_size)

    def forward(self, x, y):
        xq = self.q_w(x)
        yk = self.k_w(y)
        yv = self.v_w(y)

        attn = torch.softmax((xq @ yk.T) / self.scale, dim=-1) @ yv

        return attn


In [46]:
attn = ScaledDotProductAttention(300)

In [47]:
attn(embeddings, embeddings2).shape

torch.Size([5, 300])

### Scaled dot-product Self-Attention

<img src="attachements/second/self_attn_mask.png" width="1200"/>

### Multi-head Attention

<img src="attachements/second/multihead_attn.png" width="400"/>

## Transformer

<img src="attachements/second/transformer_architecture.webp" width="600"/>

<img src="attachements/second/attn_order.avif" width="600"/>

<img src="attachements/second/thanks_attention.png" width="1200"/>